In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Importing Libraries

In [173]:
#statistics
import tensorflow as tf
from tensorflow import keras

#data structures
import numpy as np
import PIL.Image as Image

#network
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm as c

#other
from model import create_model
import os
import h5py

print(tf.__version__) #1.14
print(keras.__version__) #2.2.4

1.14.0
2.2.4-tf


In [7]:
tf.enable_eager_execution()

# Set Hyper Parameters

In [8]:
batch_size = 100
epochs = 15
IMG_HEIGHT = 255
IMG_WIDTH = 255

# Data Preparation

In [9]:
PATH = 'ShanghaiTech/'

In [10]:
# part_a are images of densed crowd
# part_b are images of less densed crowd
part_a = os.path.join(PATH, 'part_A')
part_b = os.path.join(PATH, 'part_B')

In [11]:
part_a_train = os.path.join(part_a, 'train_data/images/')  
part_b_train = os.path.join(part_b, 'train_data/images/')  
part_a_test = os.path.join(part_a, 'test_data/images/')  
part_b_test = os.path.join(part_b, 'test_data/images/')  

In [12]:
num_a_tr = len(os.listdir(part_a_train))
num_b_tr = len(os.listdir(part_b_train))

num_a_val = len(os.listdir(part_a_test))
num_b_val = len(os.listdir(part_b_test))

total_train = num_a_tr + num_b_tr
total_val = num_a_val + num_b_val

In [13]:
print('total training A images:', num_a_tr)
print('total training B images:', num_b_tr)

print('total test A images:', num_a_val)
print('total test B images:', num_b_val)
print("--")
print("Total training images:", total_train)
print("Total test images:", total_val)

total training A images: 300
total training B images: 400
total test A images: 182
total test B images: 316
--
Total training images: 700
Total test images: 498


In [139]:
def process_path(file_path):
    label = tf.strings.regex_replace(file_path,"images",'ground-truth')
    label = tf.strings.regex_replace(label, ".jpg",".h5")
    
    return label

In [143]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [255, 255])
    
    gt_image = process_path(image_file)

    return image, gt_image

In [76]:
train_dataset = tf.data.Dataset.list_files('ShanghaiTech/*/train_data/images/*.jpg')

In [77]:
train_dataset = train_dataset.map(load, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.shuffle(400)
train_dataset = train_dataset.batch(batch_size)

In [78]:
test_dataset = tf.data.Dataset.list_files('ShanghaiTech/*/test_data/images/*.jpg')
test_dataset = test_dataset.map(load, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)

# Define functions

In [153]:
loss_object = tf.keras.losses.MeanSquaredError()

In [154]:
def loss_fn(model, input_image, gt_image):

    output = model(tf.expand_dims(input_image, 0), training=True)
    pred = np.sum(output) 
    
    gt_file = h5py.File(gt_image.numpy(),'r')
    ground_truth = np.asarray(gt_file['density'])
    ground_truth = tf.cast(ground_truth, tf.float32)
    ground_truth = np.sum(ground_truth)
    
    # mean squared error
    loss = loss_object(ground_truth, pred)
    return loss.numpy()

In [155]:
optimizer = Adam(2e-4)

In [176]:
model = create_model()

In [177]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
sequential_76 (Sequential)      (None, None, None, 6 1728        input_7[0][0]                    
__________________________________________________________________________________________________
sequential_77 (Sequential)      (None, None, None, 6 37120       sequential_76[0][0]              
__________________________________________________________________________________________________
sequential_78 (Sequential)      (None, None, None, 6 0           sequential_77[0][0]              
____________________________________________________________________________________________

In [163]:
def generate_images(model, test_input, tar):
    prediction = model(tf.expand_dims(test_input, 0), training=True)
    plt.figure(figsize=(15,15))
    tar = h5py.File(tar.numpy(),'r')
    tar = np.asarray(tar['density'])
    tar = tf.cast(tar, tf.float32)
    
    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        plt.show()

In [164]:
inp, gt_img = load('ShanghaiTech/part_A/train_data/images/IMG_10.jpg')

In [170]:
generate_images(model, inp, gt_img)

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,31,31,256] vs. shape[1] = [1,63,63,256] [Op:ConcatV2] name: concat

In [ ]:
def train_step(input_image, target):
    with tf.GradientTape() as tape:
        pred = model(tf.expand_dims(input_image, 0), training=True)

        loss = loss_fn(model, input_image, target)

    generator_gradients = gen_tape.gradient(gen_loss,
                                          generator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))


In [167]:
def fit(train_ds, epochs, test_ds):
    for epoch in range(epochs):
    start = time.time()

    # Train
    for input_image, target in train_ds:
        train_step(input_image, target)

    clear_output(wait=True)
    # Test on the same image so that the progress of the model can be 
    # easily seen.
    for example_input, example_target in test_ds.take(1):
        generate_images(generator, example_input, example_target)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))

IndentationError: expected an indented block (<ipython-input-167-ab576dc94b3c>, line 3)